In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv("Heart Prediction Quantum Dataset.csv")

In [3]:
num_features = ['Age', 'BloodPressure', 'Cholesterol', 'HeartRate', 'QuantumPatternFeature']
cat_features = ['Gender']
X = data[num_features + cat_features]
y = data['HeartDisease']

In [4]:
numerical_transformer = StandardScaler()

In [5]:
preprocessor = ColumnTransformer(
    transformers = [
        ('num', numerical_transformer, num_features)
    ]
)

In [6]:
hidden_layers = (15,6)

In [7]:
model = MLPClassifier(random_state = 10, hidden_layer_sizes = hidden_layers, max_iter = 10000)

In [8]:
pipeline = Pipeline(steps = [('preprocessor', preprocessor), ('model', model)])

In [9]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['Age', 'BloodPressure',
                                                   'Cholesterol', 'HeartRate',
                                                   'QuantumPatternFeature'])])),
                ('model',
                 MLPClassifier(hidden_layer_sizes=(15, 6), max_iter=10000,
                               random_state=10))])

In [10]:
pipeline.score(X,y)

0.94

In [11]:
kfolds = 10

In [12]:
scores = cross_val_score(pipeline, X, y, scoring='roc_auc_ovr', cv=kfolds)

In [13]:
mean_score = np.mean(scores)

In [14]:
std_score = np.std(scores)

In [15]:
'''
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

mlp = MLPClassifier(random_state=10, max_iter=10000)

pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('model', mlp)
])

l1_vals = list(range(10, 31))   
l2_vals = list(range(5, 16))    
layer_pairs = [(i, j) for i in l1_vals for j in l2_vals]

param_grid = {
    'model__hidden_layer_sizes': layer_pairs
}

gs = GridSearchCV(
    estimator=pipe,
    param_grid=param_grid,
    scoring='roc_auc_ovr',
    cv=10,
    n_jobs=-1,
    refit=True,
    return_train_score=False
)

gs.fit(X, y)

print("Mejor AUC (CV): " + str(round(gs.best_score_, 4)))
print("Mejor configuración de capas: " + str(gs.best_params_['model__hidden_layer_sizes']))

best_pipe = gs.best_estimator_
print("Score in-sample (accuracy): " + str(round(best_pipe.score(X, y), 4)))

cvres = pd.DataFrame(gs.cv_results_)
cvres = cvres[['mean_test_score', 'param_model__hidden_layer_sizes', 'rank_test_score']].sort_values('rank_test_score')
print(cvres.head(10))

'''

'\nimport numpy as np\nimport pandas as pd\nfrom sklearn.neural_network import MLPClassifier\nfrom sklearn.pipeline import Pipeline\nfrom sklearn.model_selection import GridSearchCV\n\nmlp = MLPClassifier(random_state=10, max_iter=10000)\n\npipe = Pipeline([\n    (\'preprocessor\', preprocessor),\n    (\'model\', mlp)\n])\n\nl1_vals = list(range(10, 31))   \nl2_vals = list(range(5, 16))    \nlayer_pairs = [(i, j) for i in l1_vals for j in l2_vals]\n\nparam_grid = {\n    \'model__hidden_layer_sizes\': layer_pairs\n}\n\ngs = GridSearchCV(\n    estimator=pipe,\n    param_grid=param_grid,\n    scoring=\'roc_auc_ovr\',\n    cv=10,\n    n_jobs=-1,\n    refit=True,\n    return_train_score=False\n)\n\ngs.fit(X, y)\n\nprint("Mejor AUC (CV): " + str(round(gs.best_score_, 4)))\nprint("Mejor configuración de capas: " + str(gs.best_params_[\'model__hidden_layer_sizes\']))\n\nbest_pipe = gs.best_estimator_\nprint("Score in-sample (accuracy): " + str(round(best_pipe.score(X, y), 4)))\n\ncvres = pd.Da

In [16]:
#clase 20 oct
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

x1 = [10,10]
x2 = [20,5]
x3 = [30,15]

def function(X_hl):
    global X, y
    mlp = MLPClassifier(random_state=10, hidden_layer_sizes=X_hl, max_iter=10000)
    pipe = Pipeline([('preprocessor', preprocessor),('model', mlp)])
    pipe.fit(X,y)
    scores = cross_val_score(pipe,X, y,cv=10,scoring="roc_auc_ovr")
                  
    return np.mean(scores)


In [17]:
function([10, 10])

0.9765

In [18]:
x1 = [10, 10]
x2 = [20, 5]
x3 = [30, 15]
X_hl = np.array([x1, x2, x3], dtype=int)

scores = []
for x in X_hl:
    mean_score = function(tuple(x))   
    scores.append(mean_score)

scores = np.array(scores, dtype=float)
kernel = RBF(length_scale=5.0)
regresion = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=5, normalize_y=True, random_state=42)
regresion.fit(X_hl, scores)



GaussianProcessRegressor(kernel=RBF(length_scale=5), n_restarts_optimizer=5,
                         normalize_y=True, random_state=42)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from skopt import BayesSearchCV
from skopt.space import Real

In [ ]:
df = pd.read_csv("brain_tumor_dataset.csv")
target_col = "Tumor_Type"
num_features = df.select_dtypes(include=["int64", "float64"]).columns.tolist()
cat_features = df.select_dtypes(include=["object"]).columns.tolist()
if target_col in num_features:
    num_features.remove(target_col)
if target_col in cat_features:
    cat_features.remove(target_col)

In [ ]:
X = df.drop(columns=[target_col])
y = df[target_col].values.ravel()   

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_features),
        ("cat", categorical_transformer, cat_features)
    ],
    remainder="drop"
)

svc = SVC(kernel="linear", probability=True, random_state=42)

pipeline = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("svc", svc)
])


In [ ]:
cv_strategy = KFold(n_splits=10, shuffle=True, random_state=42)

# 7) Métrica base con CV
scores = cross_val_score(
    pipeline,
    X, y,
    cv=cv_strategy,
    scoring="roc_auc_ovr",
    n_jobs=-1
)
print("ROC AUC OVR (media CV, modelo base):", np.mean(scores))

In [ ]:
search_spaces = {
    "svc__C": Real(1e-4, 1e4, prior="log-uniform")
}

opt = BayesSearchCV(
    estimator=pipeline,
    search_spaces=search_spaces,
    n_iter=50,                  # número de iteraciones de búsqueda
    cv=cv_strategy,
    scoring="roc_auc_ovr",
    n_jobs=-1,
    random_state=42,
    verbose=1
)

opt.fit(X, y)

print("Mejores hiperparámetros:", opt.best_params_)
print("Mejor ROC AUC OVR (CV):", opt.best_score_)


In [16]:
'''
svc_model = SVC(random_state=42, probability=True)

pipeline = Pipeline(steps=[
    ('svc', svc_model)
])

param_distributions = {

    'svc__C': sp_uniform(loc=0.1, scale=100),

    'svc__gamma': sp_uniform(loc=0.001, scale=1),

    'svc__kernel': ['linear']
}
'''

In [ ]:
'''
random_search = RandomizedSearchCV(
    pipeline,
    param_distributions,
    n_iter=20,          
    cv=5,               
    scoring='roc_auc',  
    random_state=42,
    n_jobs=-1,          
    verbose=2           
)
random_search.fit(X_train, y_train)
'''

Fitting 5 folds for each of 20 candidates, totalling 100 fits


In [ ]:
'''
best_score = random_search.best_score_
best_params = random_search.best_params_
test_score = random_search.score(X_test, y_test)

print(best_score:.4f)
print(test_score:.4f)
for param, value in best_params.items():
    print(f"  - {param}: {value:.4f}" if isinstance(value, float) else f"  - {param}: {value}")
    '''

In [37]:
'''
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel as Cst

np.random.seed(42)
'''

In [43]:
'''
df = pd.read_csv("brain_tumor_dataset.csv")
df.head()
'''

,Patient_ID,Age,Gender,Tumor_Type,Tumor_Size,Location,Histology,Stage,Symptom_1,Symptom_2,Symptom_3,Radiation_Treatment,Surgery_Performed,Chemotherapy,Survival_Rate,Tumor_Growth_Rate,Family_History,MRI_Result,Follow_Up_Required
0,1,73,Male,Malignant,5.375612,Temporal,Astrocytoma,III,Vision Issues,Seizures,Seizures,No,No,No,51.312579,0.111876,No,Positive,Yes
1,2,26,Male,Benign,4.847098,Parietal,Glioblastoma,II,Headache,Headache,Nausea,Yes,Yes,Yes,46.373273,2.165736,Yes,Positive,Yes
2,3,31,Male,Benign,5.588391,Parietal,Meningioma,I,Vision Issues,Headache,Seizures,No,No,No,47.072221,1.884228,No,Negative,No
3,4,29,Male,Malignant,1.436600,Temporal,Medulloblastoma,IV,Vision Issues,Seizures,Headache,Yes,No,Yes,51.853634,1.283342,Yes,Negative,No
4,5,54,Female,Benign,2.417506,Parietal,Glioblastoma,I,Headache,Headache,Seizures,No,No,Yes,54.708987,2.069477,No,Positive,Yes


In [45]:
'''
np.random.seed(42)

df = pd.read_csv("brain_tumor_dataset.csv")

y = df["Tumor_Type"].copy()
X = df.drop(columns=["Tumor_Type"]).copy()
X = pd.get_dummies(X, drop_first=True)

num_cols = X.columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[("num", StandardScaler(), num_cols)],
    remainder="drop"
)
def evaluate_arch(hidden_tuple):
    clf = MLPClassifier(
        hidden_layer_sizes=hidden_tuple,
        activation="relu",
        solver="adam",
        alpha=0.0001,
        learning_rate="adaptive",
        max_iter=10000,
        random_state=42
    )
    pipe = Pipeline(steps=[("prep", preprocessor), ("clf", clf)])
    scores = cross_val_score(
        pipe, X, y,
        cv=10,
        scoring="roc_auc_ovr" if y.nunique() > 2 else "roc_auc"
    )
    return float(np.mean(scores))
'''

In [51]:
'''
X_obs = []
y_obs = []

seed_points = [(10,10), (20,5), (30,15)]
for hp in seed_points:
    auc = evaluate_arch(hp)
    X_obs.append([hp[0], hp[1]])
    y_obs.append(auc)

X_obs = np.array(X_obs, dtype=float)   
y_obs = np.array(y_obs, dtype=float)   

seed_tbl = pd.DataFrame(
    np.column_stack([X_obs, y_obs]),
    columns=["h1", "h2", "auc"]
)
print(seed_tbl)
'''

     h1    h2       auc
0  10.0  10.0  0.487209
1  20.0   5.0  0.499363
2  30.0  15.0  0.496060


In [53]:
'''
def make_features(h_mat):
    h1 = h_mat[:,0].reshape(-1,1)
    h2 = h_mat[:,1].reshape(-1,1)
    Phi = np.hstack([h1, h2, h1*h1, h2*h2, h1*h2])
    return Phi

surrogate = LinearRegression()
'''

In [ ]:
'''
h1_space = np.arange(5, 61, 1)   
h2_space = np.arange(5, 61, 1)   
grid = np.array([(i, j) for i in h1_space for j in h2_space], dtype=float)

n_iter = 20        
explore_prob = 0.2

history = []
for t in range(n_iter):
    Phi = make_features(X_obs)
    surrogate.fit(Phi, y_obs)

    if np.random.rand() < explore_prob:
        x_next = grid[np.random.randint(0, grid.shape[0])].reshape(1, -1)
    else:
        Phi_grid = make_features(grid)
        pred = surrogate.predict(Phi_grid)
        best_idx = int(np.argmax(pred))
        x_next = grid[best_idx].reshape(1, -1)

    while any((X_obs == x_next).all(axis=1)):
        x_next = grid[np.random.randint(0, grid.shape[0])].reshape(1, -1)

    h1_val = int(x_next[0,0])
    h2_val = int(x_next[0,1])
    auc_next = evaluate_arch((h1_val, h2_val))

    X_obs = np.vstack([X_obs, x_next])
    y_obs = np.append(y_obs, auc_next)

    best_so_far = float(np.max(y_obs))
    history.append(best_so_far)

    print("Iteración", t+1, "-> h1=", h1_val, "h2=", h2_val, "AUC=", round(auc_next,5), "Best=", round(best_so_far,5))
'''

Iteración 1 -> h1= 60 h2= 5 AUC= 0.49202 Best= 0.49936
Iteración 2 -> h1= 60 h2= 18 AUC= 0.4933 Best= 0.49936
Iteración 3 -> h1= 37 h2= 7 AUC= 0.50293 Best= 0.50293
Iteración 4 -> h1= 35 h2= 5 AUC= 0.4993 Best= 0.50293
Iteración 5 -> h1= 13 h2= 23 AUC= 0.50442 Best= 0.50442
